In [1]:
%pip install numpy matplotlib torch scikit-image opencv-python


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.9 MB 1.3 MB/s eta 0:00:10
   - -------------------------------------- 0.5/12.9 MB 1.3 MB/s eta 0:00:10
   -- ------------------------------------- 0.8/12.9 MB 1.0 MB/s eta 0:00:12
   --- ------------------------------------ 1.0/12.9 MB 967.3 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/12.9 MB 967.3 kB/s eta 0:00:13
   ---- ----------------------------------- 1.3/12.9 MB 894.7 kB/s eta 0:00:13
   ----- ---------------------------------- 1.8/12.9 MB 1.1 MB/s eta 0:00:11
   ----- ---------------------------------- 1.8/12.9 MB 1.1 MB/s eta 0:00:11
   ------ ----------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
from skimage.metrics import structural_similarity as ssim
from math import sqrt, log10
import matplotlib.pyplot as plt
from time import time


In [5]:
class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()

        self.frontend = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True)
        )

        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, dilation=2, padding=2), nn.ReLU(inplace=True)
        )

        # IMPORTANT: must match Google Colab name!
        self.output = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output(x)
        return x


In [6]:
model = CSRNet()
model.load_state_dict(torch.load("csrnet_final.pth", map_location="cpu"))
model.eval()

print("CSRNet model loaded successfully.")


CSRNet model loaded successfully.


In [10]:
def compute_mae_rmse(pred_count, gt_count):
    err = pred_count - gt_count
    return abs(err), err**2



In [13]:
from tqdm import tqdm

img_dir = "./processed/train/images"
den_dir = "./processed/train/density"

mae_total = 0
mse_total = 0
times = []

files = sorted(os.listdir(img_dir))

# Progress bar added here ↓↓↓
for fname in tqdm(files, desc="Evaluating Part A", unit="image"):

    # Load sample
    img = np.load(os.path.join(img_dir, fname))      # H,W,3
    gt = np.load(os.path.join(den_dir, fname))       # 64x64

    img_tensor = torch.from_numpy(img).permute(2,0,1).unsqueeze(0).float()

    start = time()
    with torch.no_grad():
        pred = model(img_tensor).squeeze().numpy()
    end = time()

    pred_count = pred.sum()
    gt_count = gt.sum()

    # Metrics
    mae, mse = compute_mae_rmse(pred_count, gt_count)

    mae_total += mae
    mse_total += mse
    times.append(end - start)


Evaluating Part A: 100%|██████████| 300/300 [13:06<00:00,  2.62s/image]


In [14]:
N = len(files)

MAE = mae_total / N
RMSE = sqrt(mse_total / N)
Avg_Time = sum(times) / N

print("\n===== MODEL EVALUATION RESULTS =====")
print(f"MAE  (Mean Absolute Error)      : {MAE:.2f}")
print(f"RMSE (Root Mean Squared Error)  : {RMSE:.2f}")
print(f"Avg Prediction Time per Image    : {Avg_Time:.4f} sec")
print("====================================")



===== MODEL EVALUATION RESULTS =====
MAE  (Mean Absolute Error)      : 26.53
RMSE (Root Mean Squared Error)  : 49.02
Avg Prediction Time per Image    : 2.4388 sec
